In [8]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%%html
<style type='text/css'>
.CodeMirror{
    font-size: 16px;
}

div.output_area pre {
    font-size: 16px;
}
</style>

# Trenamento do modelos

Neste notebook foram feitos treinamentos de modelos a partir dos dados gerados anteriormente e salvos em arquivos. O processo foi dividido em dus etapas:

- **Etapa 1:**  treinamentos dos modelos manualmente e individualmente para os dados sem nenhuma transformação. O objetivo dest etapa é testar algumas configurações para os jyperparâmetros visando encontrar uma melhor configuração e entender o compartamento dos modelos. nesta etapa pode ser definido os intervalos de hiperparametros para um processo de optimização. Neste trabalho os valores encontrados como optimos foram estabelecidos para os treinamentos com dados alterados
</br>

- **Etapa 2:**  foi feito o treinamento para os dados com alterações.. O treinamento foi feito automaticamente para cada caso por meio de um loop. Na final desta etapa os resultados podem ser comparados


Nesta seção feitos alguns treinamentos de modelos visando obter o que fornace o melhor resultado

Considerei 5 modelos para o estudo () cada modelo foi treinado considerando com as seguintes tranformações nos dados:


OBS: nenhuma transformação é aplicada na variável alvo

Sequência:

- Divide os dados
- Faz uma transformação
- Treina o modelo
- Testa o modelo

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# transfomações
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import Normalizer

# para divisao dos dados
from sklearn.model_selection import train_test_split

# modelos
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

# metricas
from sklearn import metrics

# para não mostrar mensagens com warnings
import warnings

import time

In [14]:
#leitura dos dados
df = pd.read_csv(".//dados//dados_originais//dados_treinamento.csv" ,sep=',');
df.head(3)

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
0,24.0,M,167.3,56.10,15.6,76.0,130.0,44.6,23.1,44.0,208.0,C
1,54.0,F,161.0,51.86,29.9,88.0,154.0,22.1,23.0,36.0,148.0,A
2,34.0,M,171.7,74.22,22.6,86.0,138.0,56.1,11.4,38.0,229.0,C


In [15]:
#Mapeia a variável 'gender' para valores inteiros
gender_map = {'M': 0, 'F': 1}
var = 'gender'
df[var] = df[var].map(gender_map)

In [16]:
# seleciona variaveis de treinamento e alvo
var_train=[var for var in df.columns if var != 'class'] 

X = df[var_train]   
y = df['class'] 

# divide os dados
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.1,
                                                    random_state=0)

In [17]:
transformacoes={
    'sem_transformacao': [],     #dados sem tranformação
    'yeo_johnson': [PowerTransformer(method="yeo-johnson")],
    'quantile': [QuantileTransformer(output_distribution="normal")],
    'standart': [StandardScaler()],
    'normalization': [Normalizer()],
}

In [18]:
classificadores={'decisionTree':DecisionTreeClassifier(criterion="entropy",
                               max_depth=24,
                               min_samples_split=2,
                               min_samples_leaf=1,
                               min_impurity_decrease=0,
                               random_state=0),
                
                'randomForest':RandomForestClassifier(n_estimators=50,
                             max_depth=24,
                             max_features=5,
                             bootstrap=True,
                             max_samples=None,
                             random_state=0) ,
                 
                 'kNearestNeighbour':KNeighborsClassifier(n_neighbors=50,
                                                        #weights='distance'
                                                        ),
                                                          
                 'rbf_SVM':svm.SVC(kernel="rbf",random_state=0,
                        gamma=2.9,                
                        max_iter=1000,
                        C=0.1),
                                                          
                 'poly3_SVM':svm.SVC(kernel="poly",
                                     degree=5,
                                     random_state=0,
                                     gamma=2.9,                
                                     max_iter=5000,
                                     C=0.5)
                }
                

In [20]:
#variáveis que serão transformadas
listaVar=['age',
 'height_cm',
 'weight_kg',
 'body fat_%',
 'diastolic',
 'systolic',
 'gripForce',
 'sit and bend forward_cm',
 'sit-ups counts',
 'broad jump_cm'
  ]

In [21]:
warnings.filterwarnings("ignore")

dict_metricas={}

for key in transformacoes.keys():
    
    print("--- Treinamento para dados com a transformação: "+key)
    t_ini=time.time()
    
    # copia os dados
    X_train_temp = X_train.copy()
    X_test_temp = X_test.copy()     
        
        
    # transforma    
    if transformacoes[key]:        
        dados_train = X_train_temp[listaVar].values  
        dados_test = X_test_temp[listaVar].values
        
        obj = transformacoes[key][0].fit(dados_train)  
        
        X_train_temp[listaVar] = obj.transform(dados_train)  
        X_test_temp[listaVar] = obj.transform(dados_test)  


    #treina os modelos
    metricas_modelo=[]
    for nome_modelo, clf in classificadores.items():    
        
        clf.fit(X_train_temp,y_train)        
        y_hat = clf.predict(X_test_temp)
        
        # resultados
        metricas_modelo= metricas_modelo + [metrics.accuracy_score(y_test, y_hat)]
        
        print(nome_modelo+" treinado!")
        
    
    
    # armazena os resultados
    dict_metricas[key]=metricas_modelo    
    print("(tempo: "+str(time.time()-t_ini)+" seg.)\n")    

    

--- Treinamento para dados com a transformação: sem_transformacao
decisionTree treinado!
randomForest treinado!
kNearestNeighbour treinado!
rbf_SVM treinado!
poly3_SVM treinado!
(tempo: 9.487128734588623 seg.)

--- Treinamento para dados com a transformação: yeo_johnson
decisionTree treinado!
randomForest treinado!
kNearestNeighbour treinado!
rbf_SVM treinado!
poly3_SVM treinado!
(tempo: 11.349371433258057 seg.)

--- Treinamento para dados com a transformação: quantile
decisionTree treinado!
randomForest treinado!
kNearestNeighbour treinado!
rbf_SVM treinado!
poly3_SVM treinado!
(tempo: 11.152438163757324 seg.)

--- Treinamento para dados com a transformação: standart
decisionTree treinado!
randomForest treinado!
kNearestNeighbour treinado!
rbf_SVM treinado!
poly3_SVM treinado!
(tempo: 14.295898675918579 seg.)

--- Treinamento para dados com a transformação: normalization
decisionTree treinado!
randomForest treinado!
kNearestNeighbour treinado!
rbf_SVM treinado!
poly3_SVM treinado!
(te

In [23]:
# apresentação dos resultados
df=pd.DataFrame(dict_metricas)
df.index=list(classificadores.keys())
df

,sem_transformacao,yeo_johnson,quantile,standart,normalization
decisionTree,0.624378,0.621891,0.621061,0.624378,0.597015
randomForest,0.718905,0.721393,0.719735,0.720564,0.675788
kNearestNeighbour,0.576285,0.600332,0.596186,0.614428,0.581260
rbf_SVM,0.509950,0.547264,0.537313,0.560531,0.328358
poly3_SVM,0.334992,0.373964,0.338308,0.404643,0.523217
